In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data = np.loadtxt('parameters.txt')

def potentials(r):  
    rij_intermediate = r.reshape((N, 1, 3)) - r.reshape((1, N, 3))
    rij = np.sqrt((rij_intermediate**2).sum(axis = 2))
    rij[np.arange(N), np.arange(N)] = np.inf

    ri = np.sqrt((r**2).sum(axis=1))

    Vs = np.zeros_like(ri)
    conditionVs = ri > L

    
    Vs[conditionVs] = 0.5 * f * (L - ri[conditionVs])**2

    auxiliary_to_6pow = (R/rij)**6
    auxiliary_to_12pow = auxiliary_to_6pow**2
    
    Vp = e *(auxiliary_to_12pow - 2 * auxiliary_to_6pow)
    Fp = 12*e*(auxiliary_to_12pow - auxiliary_to_6pow).reshape((N,N,1)) * rij_intermediate / rij.reshape((N,N,1))**2
    Fp_summed = Fp.sum(axis=1)
    Epot = Vs.sum()+Vp.sum() * 0.5

    Fs = np.zeros_like(Fp_summed)
    Fs[conditionVs] = f * (L - ri[conditionVs, np.newaxis]) * r[conditionVs] / ri[conditionVs, np.newaxis]

    F = Fp_summed + Fs

    mod_F = np.sqrt((Fs**2).sum(axis=1))
    pres = 1/(4*np.pi*L**2)*mod_F.sum()
    return F, Epot, pres

n = data[0]
a = data[6]
m = int(data[1])
T = data[7] 
e = data[2]
R = data[3] 
f = int(data[4])
L = data[5] 
tau = data[8]
S_d = int(data[10])
S_o = int(data[9])
S_out = int(data[11])
S_xyz = int(data[12])

n = int(n)
N = n*n*n
k = 8.31e-3
i = 0
r = np.zeros((N, 3))
p = np.zeros((N, 3))

b0 = np.array([a, 0, 0])
b1 = np.array([0.5*a, 0.5*a*np.sqrt(3), 0])
b2 = np.array([0.5*a, a*np.sqrt(3)/6, a*np.sqrt(2/3)])



In [2]:
T = 1000 #set temperature in absolute units - if 0, crystal will only show gentle oscilations
for i0 in range (n):
    for i1 in range (n):
        for i2 in range (n):
            i = i0 + n*i1+n*n*i2
            r[i,:] = (i0 - 0.5*(n-1))*b0+(i1 - 0.5*(n-1))*b1+(i2 - 0.5*(n-1))*b2

energia = 1
with open('output.xyz', 'w') as f_file: # File needs to be saved this way in order to open the output file in Jmoll
    f_file.write("{}\n".format(N))      # This allows user to animate boiling of the crystal in this applet
    f_file.write("Energy =     {} kcal/mol\n".format(energia))
    
    df = pd.DataFrame(data=r, columns=list('xyz'), index=n*n*n*['Ar'])
    df.to_csv(f_file, header=False, sep=' ', float_format='%.5f')

    energy = -0.5*k*T*np.log(np.random.random((N, 3)))
    E_av_x = np.mean(energy,axis=0)
    if (E_av_x != 0).all():
        energy *=  (0.5*k*T)/E_av_x
    p = np.random.choice([-1, 1], size=(N,3))*np.sqrt(2*m*energy)

    p_cm = p.sum(axis=0)
    p -= p_cm/N

    F, Epot, pres = potentials(r)

    mean_temperature = 0
    mean_pressure = 0
    mean_Energy = 0
    s = 0
    
with open('output.xyz', 'w') as f_file:
    for i in range(int(S_d+S_o)):
        s+=1
        p = p+0.5*F*tau
        r = r+p*tau/m
        F, Epot, pres = potentials(r)
        p = p+0.5*F*tau
        Ekin = (p**2).sum(axis=1)/(2*m)
        Ekin_total = Ekin.sum()
        E = Epot + Ekin_total
        T = 2/(3*N*k)*Ekin_total
        t = i*tau

        if i % S_xyz==0: 
            
            f_file.write("{}\n".format(N))
            f_file.write("Energy =     {} kcal/mol\n".format(energia))

            df = pd.DataFrame(data=r, columns=list('xyz'), index=n*n*n*['Ar'])
            df.to_csv(f_file, header=False, sep=' ', float_format='%.5f')

        if i >= S_o:
            mean_temperature += T
            mean_pressure += pres
            mean_Energy += E
        
    
        
mean_temperature/=S_d
mean_pressure/=S_d
mean_Energy/=S_d
print("T_av", mean_temperature, "P_av", mean_pressure, "mean_Energy", mean_Energy)

T_av 597.507758544 P_av 13.3580796424 mean_Energy 886.053039319


In [92]:
# Presenting Energy(time step) plot
from pylab import *

s = 0
E_tab = []
t_tab = []

for i in range(int(S_d+S_o)):
    s+=1
    p = p+0.5*F*tau
    r = r+p*tau/m
    F, Epot, pres = potentials(r)
    p = p+0.5*F*tau
    Ekin = (p**2).sum(axis=1)/(2*m)
    Ekin_total = Ekin.sum()
    E = Epot + Ekin_total
    E_tab.append(E)
    T = 2/(3*N*k)*Ekin_total
    t = i*tau
    t_tab.append(t)
    
plot(t_tab, E_tab)
plt.title(data[7])

locs,labels = xticks()
xticks(locs, map(lambda x: "%g" % x, locs))
xlabel('time step')


locs,labels = yticks()
yticks(locs, map(lambda x: "%.1f" % x, locs))
ylabel('energy')

show()